In [3]:
import cantools

def generate_c_source_by_cycle_time(db, filename):
    messages_by_cycle_time = {}

    # Categorize messages by cycle time
    for message in db.messages:
        cycle_time = message.cycle_time if message.cycle_time is not None else 0
        if cycle_time not in messages_by_cycle_time:
            messages_by_cycle_time[cycle_time] = []
        messages_by_cycle_time[cycle_time].append(message)
    
    with open(filename, 'w') as f:
        # Include standard headers
        f.write('#include <stdint.h>\n')
        f.write('#include <string.h>\n\n')
        f.write('#include "ultralite_can_messages.h"\n\n\n')

        for cycle_time, messages in messages_by_cycle_time.items():
            f.write(f'/* Messages with cycle time {cycle_time} ms */\n\n')

            for message in messages:
                f.write(f'/* {message.name} */\n')
                f.write(f'#define {message.name.upper()}_ID 0x{message.frame_id:X}\n\n')
                

                # Generate encoding function
                f.write(f'void encode_{message.name}(const {message.name}_t* msg, uint8_t* data) {{\n')
                f.write(f'    memset(data, 0, 8);\n')
                for signal in message.signals:
                    start_bit = signal.start  # Starting bit position of the signal
                    start_byte = start_bit // 8  # Calculate the starting byte position
                    bit_offset = start_bit % 8  # Calculate the bit offset within the byte

                    if signal.length ==48:
                        for i in range(6):
                            f.write(f'    data[{start_byte + i}] |= (msg->{signal.name} >> {(i * 8) - bit_offset}) & 0xFF;\n')
                    if signal.length == 32:
                        for i in range(4):
                            f.write(f'    data[{start_byte + i}] |= (msg->{signal.name} >> {(i * 8) - bit_offset}) & 0xFF;\n')
                    elif signal.length == 16:
                        for i in range(2):
                            f.write(f'    data[{start_byte + i}] |= (msg->{signal.name} >> {(i * 8) - bit_offset}) & 0xFF;\n')
                    elif signal.length == 8:
                        f.write(f'    data[{start_byte}] |= (msg->{signal.name} & 0xFF) << {bit_offset};\n')
                    elif signal.length == 4:
                        f.write(f'    data[{start_byte}] |= (msg->{signal.name} & 0xF) << {bit_offset};\n')
                    elif signal.length == 1:
                        f.write(f'    data[{start_byte}] |= (msg->{signal.name} & 0x1) << {bit_offset};\n')
                    else:
                        # For other sizes, you would add additional handling here
                        pass
                #for signal in message.signals:
                #    start_byte = signal.start // 8
                #    f.write(f'    data[{start_byte}] = msg->{signal.name} & 0xFF;\n')
                 #   f.write(f'    data[{start_byte + 1}] = (msg->{signal.name} >> 8) & 0xFF;\n')
                f.write(f'}}\n\n')
            f.write('\n')

print("Starting conversion...")
db = cantools.database.load_file(r"D:\EM\UltraLite\UltraLite\UltraLite.dbc")
generate_c_source_by_cycle_time(db, 'ultralite_can_messages_by_cycle_time.c')
print("Conversion complete. Check the 'ultralite_can_messages_by_cycle_time.c' file.")


Starting conversion...
Conversion complete. Check the 'ultralite_can_messages_by_cycle_time.c' file.
